In [17]:
import numpy as np
import scipy as sp

In [18]:
from quadrotor.dynamics import QuadrotorDynamicsBase, QuadrotorState
from quadrotor.controller import ControllerBase, QuadrotorCommands
from quadrotor.trajectory import TrajectoryBase, TrajectoryState

In [19]:
from sym import Rot3

def dRot3(R: Rot3, omega: np.ndarray) -> Rot3:
    """
    NOTE(roy): If interested, we can discuss the math here later. There are other
    ways of doing this but to keep the ode_int formulation, we specifically require
    dq/dt. Refer to 'Baseile Graf - Quaternions and Dynamics' or (for intuition)
    https://math.stackexchange.com/questions/1896379/how-to-use-the-quaternion-derivative
    """
    quat = np.array(R.to_storage())  # Convert to raw quaternion
    (q0, q1, q2, q3) = quat  # xyzw storage, see https://symforce.org/api-gen-cpp/class/classsym_1_1Rot3.html

    G = np.array([[ q3,  q2, -q1, -q0],
                [-q2,  q3,  q0, -q1],
                [ q1, -q0,  q3, -q2]])
    quat_dot = (G.T @ omega) / 2
    return Rot3.from_storage(quat_dot)

### Building a 3D quadrotor model
In this second notebook, we'll build a full 3-dimensional quadrotor model! In this one I will be leaving you guys alone a lot more, but as discussed in the lectures, there is a lot of overlap with the 2D model we made last week, so feel free to copy your implementation from there and simply expand it here!

**When implementing things like this, I recommend working in steps!** For example, in your 2D model, you may find it's much better to _first_ model the accelerations; see whether that works and only _then_ move on to the angular acceleration. If you do both at the same time, finding bugs (and everyone introduces bugs once in a while!) is much harder. **Keep testing as you write code!**

#### Recommended steps
- Copy your 2D quadrotor model
- Work your way down the same way we did on the 3D model
    - Implement the mixing matrix (i.e. compute u1 and u2 from the rotor rates)
    - Modify the `state_derivative` functions to take a vector for u2 rather than a float (because now of course that's 3 numbers!)
    - Modify the `accel` formulation in your model to 3D (see equations in slides)
    - Modify the `angular_accel` formulation in your model to 3D (see equations in slides)

In [20]:
from dataclasses import dataclass, field

@dataclass
class Params:
    """A little utility class to hold our quadrotor parameters"""
    mass: float = 1.352  # [kg]

    # NOTE(roy) this is a matrix now as discussed!
    # NOTE(roy) don't get scared by the field/default_factory thing! It's just a way to do default field initialization for dataclasses,
    # see https://docs.python.org/3/library/dataclasses.html. Don't worry about it!
    inertia: np.ndarray = field(default_factory=lambda: np.array([
        [9.8e-3, 0, 0],
        [0, 10.02e-3, 0],
        [0, 0, 18.6e-3],
    ]))

    rotor_diameter: float = 10 * 0.0254  # [m] 10 inches

    static_thrust_coefficient: float = 0.14553  # [-] Thrust coefficient
    static_torque_coefficient: float = 0.01047  # [-] Torque coefficient

    # NOTE(roy) assume this is the same front-to-back and left-to-right
    arm_length: float = 0.3814 / 2.0  # [m] Arm length

    # Universal constants
    g: float = 9.80665  # [m/s2] Gravity
    rho: float = 1.225  # [kg/m3] Sea-level density

    def rotor_model(self, static_coefficient: float) -> float:
        # NOTE(roy) I'm just including this for completeness sake, this means we can e.g. see how air density
        # influences our simulation
        # Refer to this source for derivation: https://web.mit.edu/16.unified/www/FALL/thermodynamics/notes/node86.html
        return self.rho * static_coefficient * (self.rotor_diameter ** 4) / (4 * np.pi ** 2)

    # NOTE(roy) if you're unsure what @property decorators do: they make it so that we can call k_thrust as if it was just a class
    # variable. I.e. we'd write params.k_thrust now, rather than params.k_thrust()
    @property
    def k_thrust(self):
        return self.rotor_model(self.static_thrust_coefficient)

    @property
    def k_torque(self):
        return self.rotor_model(self.static_torque_coefficient)

# Simply treat as a global
p = Params()

In [9]:
# NOTE(roy) feel free to copy large parts from your previous assignment!
class FullQuadrotorDynamics(QuadrotorDynamicsBase):
    def __init__(self) -> None:
        super().__init__()

This code is largely the same, I set the rotor rates to 300 rad/s now, you should see it be very close to hovering with these numbers! (As I hope you found in the previous assignment as well with 600rad/s!)

In [21]:
class EmptyTrajectory(TrajectoryBase):
    def eval(self, *args):
        return TrajectoryState(0.0, position=np.zeros(3))  # Trajectory not yet important

class EmptyController(ControllerBase):
    def step(self, *args) -> QuadrotorCommands:
        return QuadrotorCommands(np.array([300, 300, 300, 300]))

In [22]:
from quadrotor.simulator import SimulatorBase, SimulatorState

# This might be slow to run the first time!
from quadrotor.renderer import animate_k3d, animate_matplotlib

In [15]:
def run_and_render(initial_state: QuadrotorState, t_total: int = 0.5):
    sim = SimulatorBase(
        dt=0.01,
        dynamics=FullQuadrotorDynamics(),
        controller=EmptyController(),
        trajectory=EmptyTrajectory(),
        initial_state=initial_state,
        t_total=t_total,
    )
    
    # Run the simulator
    output = sim.simulate()

    # K3D renderer
    plot = animate_k3d(output)
    
    plot.display()
    plot.start_auto_play()

In [14]:
# NOTE(roy) play with this variable and see what happens!
initial_state = QuadrotorState(
    position=np.zeros(3),
    velocity=np.array([0, 0, 0]),
    orientation=Rot3.from_yaw_pitch_roll(0, 0, 0),
    angular_velocity=np.zeros(3),
)

run_and_render(initial_state, 5.0)

NameError: name 'run_and_render' is not defined

### Does the simulated model make any sense?
If all is well, you should see the quadrotor roughly hovering with those 600 rad/s rotor rates

Now try some experiments for yourself and describe whether the behavior is as expected or whether there is anything wrong with your model, ideas to try:
- Can you make the vehicle yaw by setting a combination of rotor rates?
- Can you make the vehicle move forward/backward left/right by correctly lowering two rotor rates each time?
- What happens if you're in hover (i.e. the 600rad/s) and disable one rotor?

### Commit your updated notebook to your fork!